# AP Test Scores & Demographics at NYC Schools
### DATA 602 Final Project
#### Dan Smilowitz

## Project Scope
This project investigates the relationships between demographics Advanced Placement (AP) exam enrollment and scores for high schools in New York City.

The data will come from annual reports from the NYC Department of Education, as hosted on NYC OpenData.  I will attempt to see if there is a statistically significant relationship between AP test score and various demographic aspects (age, race, etc.), and to visualize this relationship.

### Data Utilized
Data is taken from will come reports from the New York City Department of Education, as hosted on [NYC OpenData](https://data.cityofnewyork.us/data?browseSearch=&agency=Department%20of%20Education%20%28DOE%29), specifically:
- Reports detailing annual number of AP exams taken and passed 
- Profile of school enrollment and demographic information

## Data Acquisition & Manipulation
### AP Test Data

Data about AP exams is only available through NYC OpenData for calendar years 2010 and 2012.  These datasets are read in, cleansed, and combined to a single `pandas` DataFrame.

In [1]:
import pandas as pd
# import AP test data
ap2010 = pd.read_csv('Data/AP_2010.csv', index_col='DBN')
ap2012 = pd.read_csv('Data/AP_2012.csv', index_col='DBN')

In [2]:
# rename columns to be consistent and concise
ap2010 = ap2010.rename(columns={
    'AP Test Takers ': 'Enrollment',
    'Total Exams Taken': 'Exams',
    'Number of Exams with scores 3 4 or 5': 'Passed'
})
ap2012 = ap2012.rename(columns={
    'SCHOOL NAME': 'SchoolName',
    'Num of AP Test Takers': 'Enrollment',
    'Num of AP Total Exams Taken': 'Exams',
    'Num of AP Exams Passed': 'Passed'
})

# create combined dataframe
ap = pd.concat([ap2010, ap2012], keys=[2010, 2012])

In [3]:
# convert values to numeric
numeric_cols = ap.columns[-3::]
for col in numeric_cols:
    ap[col] = pd.to_numeric(ap[col], errors='coerce')
   
ap.dtypes

SchoolName     object
Enrollment    float64
Exams         float64
Passed        float64
dtype: object

In [4]:
# calculate exames per student and pass rate
ap['ExamsPerStudent'] = ap.Exams / ap.Enrollment
ap['PassRate'] = ap.Passed / ap.Exams

# reset year index to column for comparison
ap = ap.reset_index(level=0)
ap = ap.rename(str, {'level_0': 'Year'})
ap.head()

,Year,SchoolName,Enrollment,Exams,Passed,ExamsPerStudent,PassRate
DBN,,,,,,,
01M448,2010,UNIVERSITY NEIGHBORHOOD H.S.,39.0,49.0,10.0,1.256410,0.204082
01M450,2010,EAST SIDE COMMUNITY HS,19.0,21.0,NaN,1.105263,NaN
01M515,2010,LOWER EASTSIDE PREP,24.0,26.0,24.0,1.083333,0.923077
01M539,2010,"NEW EXPLORATIONS SCI,TECH,MATH",255.0,377.0,191.0,1.478431,0.506631
02M296,2010,High School of Hospitality Management,NaN,NaN,NaN,NaN,NaN


### Demographic Data

Demographic data is available for each school for school years 2005-2006 through 2011-2012.  Data representing gender and racial demographic percentages are gathered for analysis.  The reported numbers are reported as percentages; these are converted to decimals.  

In [5]:
# import demographic data
demo = pd.read_csv('Data/School_Demographics_2006-2012.csv', index_col='DBN')

In [6]:
# convert school year to year of AP exam
# AP exams are taken at end of school year, so use ending year
# calculating mod 10000 will provide last 4 digits of int
demo['Year'] = demo['schoolyear'] % 10000
# filter for 2010 and 2012 to match AP data
demo = demo.loc[demo['Year'].isin([2010, 2012])]

# get only relevant columns
cols_to_use = ['Year', 'total_enrollment', 'male_per', 'female_per',
               'white_per', 'black_per', 'hispanic_per', 'asian_per']
demo = demo[cols_to_use]

# rename columns for consistency
demo = demo.rename(str, {
    'total_enrollment': 'Total',
    'male_per': 'Male',
    'female_per': 'Female',
    'white_per': 'White',
    'black_per': 'Black',
    'hispanic_per': 'Hispanic',
    'asian_per': 'Asian'
})

# convert percentages to decimals
for col in ['Male', 'Female', 'White', 'Black', 'Hispanic', 'Asian']:
    demo[col] = demo[col] / 100

demo.head()

,Year,Total,Male,Female,White,Black,Hispanic,Asian
DBN,,,,,,,,
01M015,2010,208,0.596,0.404,0.029,0.322,0.567,0.077
01M015,2012,189,0.513,0.487,0.021,0.333,0.577,0.063
01M019,2010,321,0.480,0.520,0.109,0.271,0.427,0.134
01M019,2012,328,0.448,0.552,0.085,0.247,0.482,0.155
01M020,2010,594,0.530,0.470,0.020,0.099,0.598,0.258


### Merging Datasets

The `ap` and `demo` DataFrames are merged by year and the `DBN` unique school identifier to allow for analysis.  From this combined dataset, rate of enrollment in AP classes is calculated.

Male percentage is excluded -- as it is the complement of female percentage, its inclusion would not yield additional insight.

In [7]:
# merge two datasets using DBN and year
# resetting indices avoids duplicate results
ap_demo = ap.reset_index().merge(demo.reset_index(), on=['DBN', 'Year'])
# create new multi-index
ap_demo = ap_demo.set_index(['DBN', 'Year'])

In [8]:
# calculate percentage of students enrolled in AP classes
ap_demo['APEnrollment'] = ap_demo['Enrollment'] / ap_demo['Total']
# select only needed columns (i.e. percentages and school/year)
rate_cols = ['SchoolName', 'APEnrollment', 'ExamsPerStudent','PassRate',
             'Female', 'White', 'Black', 'Hispanic', 'Asian']
ap_demo = ap_demo[rate_cols]
# drop schools with no reported AP performance
ap_demo = ap_demo.dropna(axis='rows', how='all', 
                         subset=['APEnrollment', 'ExamsPerStudent', 'PassRate'])

ap_demo.head()

,,SchoolName,APEnrollment,ExamsPerStudent,PassRate,Female,White,Black,Hispanic,Asian
DBN,Year,,,,,,,,,
01M448,2010,UNIVERSITY NEIGHBORHOOD H.S.,0.073308,1.256410,0.204082,0.462,0.047,0.271,0.464,0.180
01M450,2010,EAST SIDE COMMUNITY HS,0.033989,1.105263,NaN,0.465,0.055,0.270,0.574,0.075
01M515,2010,LOWER EASTSIDE PREP,0.043557,1.083333,0.923077,0.430,0.004,0.044,0.044,0.909
01M539,2010,"NEW EXPLORATIONS SCI,TECH,MATH",0.172297,1.478431,0.506631,0.524,0.446,0.116,0.158,0.252
02M298,2010,Pace High School,0.050847,1.000000,NaN,0.542,0.075,0.322,0.424,0.140


## Analyis of Correlation
The correlation between variables are calculated for fifteen pairs of variables, representing the intersection of five independent variables of interest and three dependent variables of interest.

These independent variables are demographic characteristics of interest:
	
- Female
- White
- Black
- Hispanic
- Asian

The independent variables are AP-exam-related characteristics:

- Share of student enrolled in AP classes
- Average number of AP exams taken per AP student
- Pass rate of AP exams taken (where scores of 3 or greater represent passing)

In [9]:
# calculate correlation table and display values of interest
corr_table = ap_demo.corr()
xy_corr = corr_table[['Female', 'White', 'Black', 'Hispanic', 'Asian']][:3]
xy_corr

,Female,White,Black,Hispanic,Asian
APEnrollment,0.026803,0.388317,-0.295794,-0.229559,0.468453
ExamsPerStudent,-0.037496,0.263257,-0.172059,-0.171840,0.302416
PassRate,0.071817,0.294645,-0.443439,-0.164906,0.484654


### Visualizing Correlations
The correlations above are visualized in a heatmap using the `plotly` library below

In [10]:
import plotly.plotly as py
import plotly.graph_objs as go

trace = go.Heatmap(
        x=['Female', 'White', 'Black', 'Hispanic', 'Asian'],
        y=['AP<br>Enrollment', 'Exams<br>Per<br>Student', 'Pass<br>Rate'],
        z=xy_corr.values.tolist(),
        colorscale='Viridis'
)

fig = go.Figure(data=[trace])
fig['layout'].update(
    title='Correlation Matrix',
    width=900,
    height=600,
    autosize=False,
    yaxis=dict(ticks=''),
    xaxis=dict(ticks='')
)

py.iplot(fig, filename='correlation-heatmap')

## Visualizing Distributions

Correlations involving the `Female` and `Hispanic` independent variables have low magnitudes and are not explored graphically.  The pairings of each dependent variable with the three remaining independent variables are presented in scatterplots below; the distribution of the variables is plotted using the `bokeh` library.

In [11]:
# import bokeh modules
from bokeh.charts import Scatter, output_notebook, show
from bokeh.models import NumeralTickFormatter
from bokeh.layouts import layout
from bokeh.palettes import Accent
output_notebook()

Loading BokehJS ...

In [12]:
# reset ap_demo index to allow coloring by year
ap_demo = ap_demo.reset_index()

# designate variables for plotting
xvars = ['White', 'Black', 'Asian']
yvars = ['APEnrollment', 'ExamsPerStudent', 'PassRate']
full_y_names = {
    'APEnrollment': 'Enrollment in AP Classes', 
    'ExamsPerStudent': 'Exams Taken per AP Student',
    'PassRate': 'AP Exam Pass Rate'
}

# create array to hold plots
corrplots = []
# create plot for each desired (x,y) pair
for yvar in yvars:
    for xvar in xvars:
        # assign plot title based on variables
        plot_title = '%s vs. Percent %s' %(full_y_names[yvar], xvar)
        # create scatter plot
        p = Scatter(ap_demo, x=xvar, y=yvar, color='Year',
                    title=plot_title, xlabel=None, ylabel=None,
                    palette=Accent[3], legend=False,
                    height=300, width=300,
                    tools='pan, box_zoom, wheel_zoom, crosshair, reset')
        # format x axis as percent
        p.xaxis[0].formatter = NumeralTickFormatter(format='0%')
        # format y axis as percent except for ExamsPerStudent
        if yvar != 'ExamsPerStudent':
            p.yaxis[0].formatter = NumeralTickFormatter(format='0%')
        # store created plot
        corrplots.append(p)

# create and show grid of plots
grid = layout([
    corrplots[0:3],
    corrplots[3:6],
    corrplots[6:]
])

show(grid)

## Linear Regression

To explore if the relationship between AP-exam-related characteristics and demographic characteristics are statistically significant, simple linear regresstion is performed for each pairing of variables using the `scipy.stats` module

In [13]:
from scipy import stats

# create set of full x variables; use same yvars as correlation
xvars_full = ['Female', 'White', 'Black', 'Hispanic','Asian']
# create DataFrame to house results
models = pd.DataFrame(columns=['y', 'x', 'eq', 'p', 'R2'])
# run linear model for each (x,y) pair
for y in yvars:
    for x in xvars_full:
        # create temporary df with NaN values dropped
        lm_data = ap_demo[[x, y]].dropna()
        # run linear model
        lm = stats.linregress(lm_data[x], lm_data[y])
        # store regression equation as string
        eq = '%s = %.3f + %.3f * %s' %(y, lm.intercept, lm.slope, x)
        # create temporary df with model characteristics
        new_model = pd.DataFrame([[y, x, eq, lm.pvalue, lm.rvalue**2]],
                                columns=['y', 'x', 'eq', 'p', 'R2'])
        # add new model to df of models
        models = models.append(new_model, ignore_index=True)

models

,y,x,eq,p,R2
0,APEnrollment,Female,APEnrollment = 0.094 + 0.021 * Female,5.510814e-01,0.000718
1,APEnrollment,White,APEnrollment = 0.080 + 0.255 * White,2.463968e-19,0.150790
2,APEnrollment,Black,APEnrollment = 0.146 + -0.116 * Black,1.710348e-11,0.087494
3,APEnrollment,Hispanic,APEnrollment = 0.144 + -0.092 * Hispanic,2.290832e-07,0.052697
4,APEnrollment,Asian,APEnrollment = 0.072 + 0.301 * Asian,1.778765e-28,0.219448
5,ExamsPerStudent,Female,ExamsPerStudent = 1.349 + -0.082 * Female,4.042231e-01,0.001406
6,ExamsPerStudent,White,ExamsPerStudent = 1.262 + 0.486 * White,2.528707e-09,0.069304
7,ExamsPerStudent,Black,ExamsPerStudent = 1.375 + -0.191 * Black,1.157635e-04,0.029604
8,ExamsPerStudent,Hispanic,ExamsPerStudent = 1.392 + -0.195 * Hispanic,1.181472e-04,0.029529
9,ExamsPerStudent,Asian,ExamsPerStudent = 1.248 + 0.547 * Asian,5.704121e-12,0.091456


Insepecting the results of the linear regression, three findings are immediately obvious:
 - The relationships between gender demographics and AP enrollment and performance are **not statistically significant** at the $\alpha = 0.05$ signficance level.
 - The relationships between racial demographics and AP enrollment and performance are **all statistically significant** at the $\alpha = 0.01$ singificance level.
 - The $R^2$ values of linear models fit to these relationships are very low, showing that they have very low predictive power.

## Conclusions
### Findings

Analysis of the correlation and linear relationships between Advanced Placement exam enrollment and performance against gender and racial demographics of high schools in New York City revealed the following:
- There are weak correlations between gender composition of student populations and AP exam enrollment and performance
  - The strongest of these correlations is 0.072 (with AP exam pass rate)
  - The weakest of these correlations is 0.027 (with AP course enrollment)
- There are stonger correlations between racial composition of student populations and AP exam enrollment and performance
  - The strongest of these is 0.485 (between percent Asian and AP exam pass rate)
  - The weakest of these is 0.165 (between percent Hispanic and AP exam pass rate)
- There exist statistically significant relationships between racial composition of student populations and AP exam enrollment and performance
  - The most significant of these is the relationship between percent Asian and AP course enrollment
    - This relationship has a p-value of $1.77 \times 10^{-28}$
    - The equation describing this relationship is $$\widehat{APEnrollment} = 0.072 + 0.301 * Asian$$
  - The predictive power of these relationships is very low -- the highest $R^2$ value identified is 0.235, meaning that only 23.5% of variation in the identified dependent variable can be explained by variation in the independent variable.

### Future Areas for Investigation
The high statistical significance of the linear regressions completed compared to their low predictive power suggests that the relationship between racial demographics may not be best described in a linear fashion.  Because there is, by definition, interaction between the four racial demographics investigated -- an increase in one variable necessitates a decrease in another -- a more complex linear regression may better explain the relationships between these demographic characteristics and AP-exam-related metrics.  Investigating the scatterplots of the variables, it seems possible that there is a non-linear relationship between racial demographics and AP exam enrollment and performance.  Higher-order regression may produce models with higher predictive power.

## Works Cited

Bokeh Development Team (2014). *Bokeh: Python library for interactive visualization*. http://www.bokeh.pydata.org.

Jones E, Oliphant E, Peterson P, et al (2001). *SciPy: Open Source Scientific Tools for Python*. http://www.scipy.org/

McKinney, Wes (2010). *Data Structures for Statistical Computing in Python*. http://pandas.pydata.org/

New York City Department of Education (2014). *AP (College Board) 2010 School Level Results*. https://data.cityofnewyork.us/Education/AP-College-Board-2010-School-Level-Results/itfs-ms3e

New York City Department of Education (2014). *AP Results*. https://data.cityofnewyork.us/Education/AP-Results/9ct9-prf9

New York City Deparment of Education (2014). *School Demographics and Accountability Snapshot 2006-2012*. https://data.cityofnewyork.us/Education/School-Demographics-and-Accountability-Snapshot-20/ihfw-zy9j

Plotly Technologies Inc. (2015). *Collaborative data science*. Montréal, QC. https://plot.ly.